# Project - RNN Classification

In this notebook, we will perform a classification task using RNNs (i.e., a sequence to value prediction). We have hourly power consumption of households for 12 hours. Based on this, we will determine whether the power grid is strained (1) or not (0). 

Therefore, use the columns from `Hour 0` to `Hour 11` to predict the `target` column in the `power.csv` data set.


Hint2: Don't forget to adjust the number of neurons in the input layers correctly.

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



C:\Users\Rishabh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Read the Dataset

In [2]:
power = pd.read_csv('power.csv')

power.head()

,Hour 0,Hour 1,Hour 2,Hour 3,Hour 4,Hour 5,Hour 6,Hour 7,Hour 8,Hour 9,Hour 10,Hour 11,target
0,2.550633,2.523400,2.582333,2.541667,2.475733,2.476233,2.455800,2.447200,2.441733,3.146133,2.661733,2.576000,1
1,1.596933,1.619567,2.473733,2.731133,2.431133,2.479667,1.690200,1.332133,1.375167,1.050900,0.585900,2.651900,1
2,0.534933,0.540467,0.575367,0.526500,0.521900,0.565333,1.426467,0.602067,0.547433,0.525067,1.270300,0.393767,0
3,1.085867,0.651233,0.634600,0.653000,0.646067,0.628400,0.611067,0.612533,0.660100,0.606067,1.471867,0.834533,0
4,0.456000,0.286300,0.310833,0.250933,0.277667,0.308633,0.610400,1.563533,1.421867,3.324400,3.207567,1.425433,1


In [3]:
power.shape

(1417, 13)

# Split the Data



In [5]:
# First 1000 days are for train
train = power.iloc[:1000]

# Remaining 417 days are for test
test = power.iloc[-417:]

In [6]:
train.shape

(1000, 13)

In [7]:
test.shape

(417, 13)

# Create Input and Target values

The first 12 columns (hourly data) will be input to predict the last column (i.e., target)

In [8]:
# The first 12 columns (from 0 to 11) are inputs

train_inputs = train.iloc[:,:12]

## Add one more dimension to make it ready for RNNs

In [9]:
#Create an additional dimension for train

train_x = np.array(train_inputs).reshape(1000,12,1)

train_x.shape 

(1000, 12, 1)

## Set the target

In [10]:
# The last column is TARGET

train_target = train.iloc[:,-1]

## Repeat for TEST

In [11]:
test.shape

(417, 13)

In [12]:
# The first 12 columns are inputs

test_inputs = test.iloc[:,:12]

In [13]:
#Create an additional dimension for test

test_x = np.array(test_inputs).reshape(417,12,1)

test_x.shape 

(417, 12, 1)

In [14]:
# The last column is TARGET

test_target = test.iloc[:,-1]

In [99]:
test_target

1000    0
1001    1
1002    0
1003    0
1004    0
       ..
1412    0
1413    1
1414    0
1415    0
1416    0
Name: target, Length: 417, dtype: int64

# Build a normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [29]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[12, 1]),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [30]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_target, epochs=50,
                    validation_data=(test_x, test_target))

Epoch 1/50
32/32 [==============================] - 32s 519ms/step - loss: 0.8548 - accuracy: 0.4770 - val_loss: 0.6837 - val_accuracy: 0.5588
Epoch 2/50
32/32 [==============================] - 11s 352ms/step - loss: 0.6440 - accuracy: 0.6470 - val_loss: 0.6193 - val_accuracy: 0.6906
Epoch 3/50
32/32 [==============================] - 11s 357ms/step - loss: 0.5728 - accuracy: 0.7260 - val_loss: 0.5724 - val_accuracy: 0.7074
Epoch 4/50
32/32 [==============================] - 10s 334ms/step - loss: 0.5356 - accuracy: 0.7400 - val_loss: 0.5494 - val_accuracy: 0.7074
Epoch 5/50
32/32 [==============================] - 10s 327ms/step - loss: 0.4981 - accuracy: 0.7500 - val_loss: 0.5359 - val_accuracy: 0.7074
Epoch 6/50
32/32 [==============================] - 10s 322ms/step - loss: 0.4822 - accuracy: 0.7450 - val_loss: 0.5218 - val_accuracy: 0.7002
Epoch 7/50
32/32 [==============================] - 10s 323ms/step - loss: 0.4789 - accuracy: 0.7510 - val_loss: 0.5369 - val_accuracy: 0.7194

In [31]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5451728105545044, 0.7505995035171509]

In [33]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.55
accuracy: 75.06%


# Build a simple RNN with one layer

In [76]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [77]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [78]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_target, epochs=50,
                    validation_data=(test_x, test_target), callbacks=callback)

Epoch 1/50
32/32 [==============================] - 18s 265ms/step - loss: 0.5614 - accuracy: 0.7000 - val_loss: 0.5472 - val_accuracy: 0.7050
Epoch 2/50
32/32 [==============================] - 7s 216ms/step - loss: 0.4834 - accuracy: 0.7520 - val_loss: 0.6775 - val_accuracy: 0.6859
Epoch 3/50
32/32 [==============================] - 3s 108ms/step - loss: 0.4843 - accuracy: 0.7380 - val_loss: 0.6132 - val_accuracy: 0.6835
Epoch 4/50
32/32 [==============================] - 4s 125ms/step - loss: 0.5514 - accuracy: 0.7190 - val_loss: 0.5243 - val_accuracy: 0.7098
Epoch 5/50
32/32 [==============================] - 3s 110ms/step - loss: 0.4917 - accuracy: 0.7550 - val_loss: 0.5660 - val_accuracy: 0.6954
Epoch 6/50
32/32 [==============================] - 3s 104ms/step - loss: 0.5255 - accuracy: 0.7410 - val_loss: 0.5456 - val_accuracy: 0.7482
Epoch 7/50
32/32 [==============================] - 4s 115ms/step - loss: 0.4942 - accuracy: 0.7460 - val_loss: 0.5223 - val_accuracy: 0.7146
Epoch

In [79]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5628818273544312, 0.7242206335067749]

In [80]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.56
accuracy: 72.42%


In [81]:
# Predictions are probabilities.

predictions = model.predict(test_x)

14/14 [==============================] - 8s 3ms/step


In [82]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)

array([[1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [83]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target, np.round(predictions))

array([[120,  90],
       [ 25, 182]], dtype=int64)

# Build a simple RNN with two or more layers

In [84]:
n_steps = 12
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])


In [85]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 21s 301ms/step - loss: 0.6208 - accuracy: 0.6600 - val_loss: 0.6256 - val_accuracy: 0.6954
Epoch 2/20
32/32 [==============================] - 7s 215ms/step - loss: 0.5443 - accuracy: 0.7280 - val_loss: 0.6465 - val_accuracy: 0.6978
Epoch 3/20
32/32 [==============================] - 4s 143ms/step - loss: 0.5177 - accuracy: 0.7420 - val_loss: 0.6237 - val_accuracy: 0.6715
Epoch 4/20
32/32 [==============================] - 5s 148ms/step - loss: 0.5805 - accuracy: 0.6900 - val_loss: 0.5329 - val_accuracy: 0.7074
Epoch 5/20
32/32 [==============================] - 4s 124ms/step - loss: 0.5006 - accuracy: 0.7530 - val_loss: 0.5718 - val_accuracy: 0.7002
Epoch 6/20
32/32 [==============================] - 6s 178ms/step - loss: 0.5147 - accuracy: 0.7430 - val_loss: 0.5123 - val_accuracy: 0.7026
Epoch 7/20
32/32 [==============================] - 5s 174ms/step - loss: 0.5134 - accuracy: 0.7320 - val_loss: 0.4960 - val_accuracy: 0.7578
Epoch

In [45]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5238609910011292, 0.7458033561706543]

In [86]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.56
accuracy: 72.42%


# Build a LSTM with one layer

In [87]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(12, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [88]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 26s 316ms/step - loss: 0.5913 - accuracy: 0.6810 - val_loss: 0.5722 - val_accuracy: 0.6906
Epoch 2/20
32/32 [==============================] - 7s 241ms/step - loss: 0.4974 - accuracy: 0.7500 - val_loss: 0.5151 - val_accuracy: 0.7434
Epoch 3/20
32/32 [==============================] - 4s 130ms/step - loss: 0.4758 - accuracy: 0.7540 - val_loss: 0.5370 - val_accuracy: 0.7458
Epoch 4/20
32/32 [==============================] - 4s 142ms/step - loss: 0.4909 - accuracy: 0.7540 - val_loss: 0.5187 - val_accuracy: 0.7146
Epoch 5/20
32/32 [==============================] - 4s 129ms/step - loss: 0.4721 - accuracy: 0.7540 - val_loss: 0.5095 - val_accuracy: 0.7242
Epoch 6/20
32/32 [==============================] - 4s 131ms/step - loss: 0.4751 - accuracy: 0.7500 - val_loss: 0.5081 - val_accuracy: 0.7410
Epoch 7/20
32/32 [==============================] - 4s 143ms/step - loss: 0.4667 - accuracy: 0.7530 - val_loss: 0.5276 - val_accuracy: 0.7530
Epoch

In [89]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5150845050811768, 0.7170263528823853]

In [90]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.52
accuracy: 71.70%


# Build a LSTM with two or more layers

In [91]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(12, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(12, return_sequences=True),
    keras.layers.LSTM(12),
    keras.layers.Dense(1, activation='sigmoid')
])

In [92]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 32s 378ms/step - loss: 0.6447 - accuracy: 0.6300 - val_loss: 0.5863 - val_accuracy: 0.7074
Epoch 2/20
32/32 [==============================] - 8s 257ms/step - loss: 0.5475 - accuracy: 0.7220 - val_loss: 0.5439 - val_accuracy: 0.7122
Epoch 3/20
32/32 [==============================] - 5s 148ms/step - loss: 0.5233 - accuracy: 0.7410 - val_loss: 0.5383 - val_accuracy: 0.7338
Epoch 4/20
32/32 [==============================] - 6s 187ms/step - loss: 0.5153 - accuracy: 0.7420 - val_loss: 0.5243 - val_accuracy: 0.7434
Epoch 5/20
32/32 [==============================] - 5s 164ms/step - loss: 0.4861 - accuracy: 0.7540 - val_loss: 0.5718 - val_accuracy: 0.7026
Epoch 6/20
32/32 [==============================] - 5s 151ms/step - loss: 0.5259 - accuracy: 0.7400 - val_loss: 0.5582 - val_accuracy: 0.7026
Epoch 7/20
32/32 [==============================] - 4s 141ms/step - loss: 0.4959 - accuracy: 0.7380 - val_loss: 0.5089 - val_accuracy: 0.7602
Epoch

In [93]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5076159834861755, 0.7242206335067749]

In [94]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.51
accuracy: 72.42%


# Build a GRU with one layer

In [58]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(12, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [59]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 24s 334ms/step - loss: 0.6680 - accuracy: 0.6090 - val_loss: 0.5965 - val_accuracy: 0.7050
Epoch 2/20
32/32 [==============================] - 7s 241ms/step - loss: 0.5057 - accuracy: 0.7480 - val_loss: 0.5394 - val_accuracy: 0.7074
Epoch 3/20
32/32 [==============================] - 4s 137ms/step - loss: 0.4844 - accuracy: 0.7490 - val_loss: 0.5238 - val_accuracy: 0.7602
Epoch 4/20
32/32 [==============================] - 4s 132ms/step - loss: 0.4893 - accuracy: 0.7580 - val_loss: 0.5092 - val_accuracy: 0.7218
Epoch 5/20
32/32 [==============================] - 4s 136ms/step - loss: 0.4684 - accuracy: 0.7550 - val_loss: 0.5128 - val_accuracy: 0.7194
Epoch 6/20
32/32 [==============================] - 4s 141ms/step - loss: 0.4790 - accuracy: 0.7470 - val_loss: 0.5056 - val_accuracy: 0.7314
Epoch 7/20
32/32 [==============================] - 4s 133ms/step - loss: 0.4690 - accuracy: 0.7530 - val_loss: 0.5255 - val_accuracy: 0.7578
Epoch

In [60]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5002162456512451, 0.7505995035171509]

In [61]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.50
accuracy: 75.06%


# Build a GRU with two or more layers

In [95]:
n_steps = 12
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [96]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_target, epochs=20,
                   validation_data = (test_x, test_target), callbacks=callback)

Epoch 1/20
32/32 [==============================] - 31s 331ms/step - loss: 0.6080 - accuracy: 0.6800 - val_loss: 0.5458 - val_accuracy: 0.7146
Epoch 2/20
32/32 [==============================] - 7s 215ms/step - loss: 0.4960 - accuracy: 0.7440 - val_loss: 0.5113 - val_accuracy: 0.7362
Epoch 3/20
32/32 [==============================] - 4s 130ms/step - loss: 0.4782 - accuracy: 0.7450 - val_loss: 0.5425 - val_accuracy: 0.7530
Epoch 4/20
32/32 [==============================] - 7s 219ms/step - loss: 0.5267 - accuracy: 0.7350 - val_loss: 0.5152 - val_accuracy: 0.7626
Epoch 5/20
32/32 [==============================] - 9s 289ms/step - loss: 0.4654 - accuracy: 0.7590 - val_loss: 0.5322 - val_accuracy: 0.7170
Epoch 6/20
32/32 [==============================] - 7s 229ms/step - loss: 0.4955 - accuracy: 0.7450 - val_loss: 0.5055 - val_accuracy: 0.7194
Epoch 7/20
32/32 [==============================] - 6s 202ms/step - loss: 0.4643 - accuracy: 0.7570 - val_loss: 0.4968 - val_accuracy: 0.7410
Epoch

In [97]:
# evaluate the model

scores = model.evaluate(test_x, test_target, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5316129326820374, 0.7194244861602783]

In [98]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.53
accuracy: 71.94%
